In [1]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(42)


In [2]:
COMPANIES = ["Waterstones", "WHSmith"]

ENGLAND_REGIONS = [
    "London", "South East", "South West", "East of England",
    "West Midlands", "East Midlands", "North West", "North East", "Yorkshire and the Humber"
]

CITY_TYPES = ["Major City", "Large Town", "Small Town", "Travel Hub"]

TITLES = [
    "The Hunt for Red October",
    "Patriot Games",
    "Clear and Present Danger",
    "The Sum of All Fears",
    "Without Remorse",
    "Red Storm Rising",
    "The Cardinal of the Kremlin",
    "Executive Orders",
    "Rainbow Six",
    "Debt of Honor",
    "The Bear and the Dragon",
    "Tom Clancy's: Power and Empire",
    "Tom Clancy's: Commander-in-Chief",
    "Tom Clancy's: Flash Point",
]

FORMATS = ["Paperback", "Hardback", "Mass Market", "Audiobook", "Ebook Card"]

AGE_BANDS = ["16-24", "25-34", "35-44", "45-54", "55-64", "65+"]
GENDERS = ["Male", "Female", "Non-binary/Other"]
INCOME_BANDS = ["Low", "Mid", "High"]
CUSTOMER_TYPES = ["New", "Returning", "Collector"]


In [3]:
def pick_weighted(options, weights, size):
    weights = np.array(weights, dtype=float)
    weights = weights / weights.sum()
    return rng.choice(options, size=size, replace=True, p=weights)


In [4]:
def company_citytype_weights(company):
    # Waterstones: more destination bookshops in cities/towns
    # WHSmith: more travel hubs + high street
    if company == "Waterstones":
        return [0.40, 0.35, 0.20, 0.05]  # Major City, Large Town, Small Town, Travel Hub
    else:
        return [0.18, 0.30, 0.22, 0.30]

def company_customer_mix(company):
    # Waterstones sees more collectors + returning readers
    # WHSmith sees more new/impulse buyers
    if company == "Waterstones":
        return [0.25, 0.60, 0.15]  # New, Returning, Collector
    else:
        return [0.45, 0.50, 0.05]


In [5]:
def base_price_by_format(fmt):
    return {
        "Mass Market": 9.99,
        "Paperback": 10.99,
        "Hardback": 22.00,
        "Audiobook": 19.99,
        "Ebook Card": 12.99,
    }[fmt]

def discount_rate(company, fmt, city_type):
    base = 0.05
    if company == "Waterstones":
        base += 0.03 if fmt in ["Paperback", "Mass Market"] else 0.01
        base += 0.01 if city_type in ["Major City", "Large Town"] else 0.00
    else:
        base += 0.06 if city_type == "Travel Hub" else 0.04
        base += 0.02 if fmt in ["Paperback", "Mass Market"] else 0.01
    return float(np.clip(base + rng.normal(0, 0.01), 0.00, 0.20))

def cogs_rate(fmt):
    # cost-of-goods share of net revenue
    return {
        "Mass Market": 0.62,
        "Paperback": 0.60,
        "Hardback": 0.55,
        "Audiobook": 0.50,
        "Ebook Card": 0.45,
    }[fmt]

def region_demand_multiplier(region):
    return {
        "London": 1.25,
        "South East": 1.10,
        "North West": 1.05,
        "Yorkshire and the Humber": 1.00,
        "East of England": 1.00,
        "West Midlands": 0.98,
        "South West": 0.95,
        "East Midlands": 0.92,
        "North East": 0.90
    }[region]

def seasonality_multiplier(week_of_year):
    # holiday surge
    if week_of_year in list(range(48, 53)) + [1, 2]:
        return 1.30
    return 1.0

def expected_units(company, city_type, customer_type, fmt, age_band):
    fmt_base = {
        "Mass Market": 1.8,
        "Paperback": 1.6,
        "Hardback": 0.8,
        "Audiobook": 0.4,
        "Ebook Card": 0.5,
    }[fmt]

    if company == "Waterstones":
        channel = 1.05 if city_type != "Travel Hub" else 0.70
    else:
        channel = 1.20 if city_type == "Travel Hub" else 0.90

    cust = {"New": 0.95, "Returning": 1.10, "Collector": 0.75}[customer_type]
    if company == "Waterstones" and customer_type == "Collector":
        cust = 0.90

    age = {"16-24": 0.60, "25-34": 0.85, "35-44": 1.00, "45-54": 1.15, "55-64": 1.25, "65+": 1.10}[age_band]

    lam = fmt_base * channel * cust * age
    return max(lam, 0.05)


In [6]:
def generate_store_master(n_stores=260):
    company = pick_weighted(COMPANIES, [0.55, 0.45], n_stores)

    regions = pick_weighted(
        ENGLAND_REGIONS,
        [0.16, 0.14, 0.09, 0.10, 0.10, 0.08, 0.13, 0.07, 0.13],
        n_stores
    )

    city_type = []
    for c in company:
        city_type.append(pick_weighted(CITY_TYPES, company_citytype_weights(c), 1)[0])

    store_ids = [f"ENG-{i:04d}" for i in range(1, n_stores + 1)]

    size = []
    for c in company:
        if c == "Waterstones":
            size.append(pick_weighted(["Small", "Medium", "Large"], [0.25, 0.55, 0.20], 1)[0])
        else:
            size.append(pick_weighted(["Small", "Medium", "Large"], [0.45, 0.50, 0.05], 1)[0])

    store_master = pd.DataFrame({
        "store_id": store_ids,
        "company": company,
        "region": regions,
        "city_type": city_type,
        "store_size": size
    })

    store_master["store_size_mult"] = store_master["store_size"].map({"Small": 0.80, "Medium": 1.00, "Large": 1.25})
    return store_master

store_master = generate_store_master(260)
store_master.head()


,store_id,company,region,city_type,store_size,store_size_mult
0,ENG-0001,WHSmith,London,Travel Hub,Small,0.80
1,ENG-0002,Waterstones,North East,Major City,Large,1.25
2,ENG-0003,WHSmith,South West,Major City,Medium,1.00
3,ENG-0004,WHSmith,London,Small Town,Medium,1.00
4,ENG-0005,Waterstones,Yorkshire and the Humber,Major City,Large,1.25


In [7]:
def generate_sales(store_master, weeks=52, year=2025, rows_per_store_week=28):
    records = []
    week_numbers = np.arange(1, weeks + 1)

    for _, s in store_master.iterrows():
        company = s["company"]
        region = s["region"]
        city_type = s["city_type"]
        size_mult = s["store_size_mult"]

        for w in week_numbers:
            season_mult = seasonality_multiplier(int(w))
            region_mult = region_demand_multiplier(region)

            titles = pick_weighted(TITLES, np.ones(len(TITLES)), rows_per_store_week)
            fmts = pick_weighted(FORMATS, [0.38, 0.10, 0.30, 0.10, 0.12], rows_per_store_week)

            age = pick_weighted(AGE_BANDS, [0.06, 0.12, 0.18, 0.24, 0.24, 0.16], rows_per_store_week)
            gender = pick_weighted(GENDERS, [0.62, 0.35, 0.03], rows_per_store_week)

            if region == "London":
                income = pick_weighted(INCOME_BANDS, [0.18, 0.52, 0.30], rows_per_store_week)
            elif region in ["South East", "East of England"]:
                income = pick_weighted(INCOME_BANDS, [0.22, 0.56, 0.22], rows_per_store_week)
            else:
                income = pick_weighted(INCOME_BANDS, [0.30, 0.58, 0.12], rows_per_store_week)

            cust_type = pick_weighted(CUSTOMER_TYPES, company_customer_mix(company), rows_per_store_week)

            for i in range(rows_per_store_week):
                fmt = fmts[i]
                title = titles[i]
                age_band = age[i]
                customer_type = cust_type[i]

                list_price = base_price_by_format(fmt)
                disc = discount_rate(company, fmt, city_type)
                net_price = list_price * (1 - disc)

                lam = expected_units(company, city_type, customer_type, fmt, age_band)
                lam *= season_mult * region_mult * size_mult

                units = rng.poisson(lam=lam)

                revenue = units * net_price
                cogs = revenue * cogs_rate(fmt)
                gross_profit = revenue - cogs

                records.append({
                    "year": year,
                    "week_of_year": int(w),
                    "store_id": s["store_id"],
                    "company": company,
                    "region": region,
                    "city_type": city_type,
                    "store_size": s["store_size"],
                    "title": title,
                    "format": fmt,
                    "age_band": age_band,
                    "gender": gender[i],
                    "income_band": income[i],
                    "customer_type": customer_type,
                    "list_price_gbp": round(list_price, 2),
                    "discount_rate": round(float(disc), 4),
                    "net_price_gbp": round(float(net_price), 2),
                    "units_sold": int(units),
                    "revenue_gbp": round(float(revenue), 2),
                    "cogs_gbp": round(float(cogs), 2),
                    "gross_profit_gbp": round(float(gross_profit), 2),
                })

    return pd.DataFrame.from_records(records)

sales = generate_sales(store_master, weeks=52, year=2025, rows_per_store_week=28)
sales.head()


,year,week_of_year,store_id,company,region,city_type,store_size,title,format,age_band,gender,income_band,customer_type,list_price_gbp,discount_rate,net_price_gbp,units_sold,revenue_gbp,cogs_gbp,gross_profit_gbp
0,2025,1,ENG-0001,WHSmith,London,Travel Hub,Small,Patriot Games,Paperback,35-44,Male,Low,Returning,10.99,0.1288,9.57,4,38.30,22.98,15.32
1,2025,1,ENG-0001,WHSmith,London,Travel Hub,Small,Debt of Honor,Hardback,55-64,Male,Low,New,22.00,0.1237,19.28,1,19.28,10.60,8.68
2,2025,1,ENG-0001,WHSmith,London,Travel Hub,Small,Debt of Honor,Audiobook,65+,Male,Mid,New,19.99,0.1285,17.42,1,17.42,8.71,8.71
3,2025,1,ENG-0001,WHSmith,London,Travel Hub,Small,Debt of Honor,Mass Market,25-34,Female,Low,New,9.99,0.1299,8.69,3,26.08,16.17,9.91
4,2025,1,ENG-0001,WHSmith,London,Travel Hub,Small,The Hunt for Red October,Hardback,16-24,Female,Low,New,22.00,0.1186,19.39,0,0.00,0.00,0.00


In [8]:
weekly_company = (
    sales.groupby(["company", "week_of_year"], as_index=False)
         .agg(units_sold=("units_sold", "sum"),
              revenue_gbp=("revenue_gbp", "sum"),
              gross_profit_gbp=("gross_profit_gbp", "sum"))
)

weekly_company.head()


,company,week_of_year,units_sold,revenue_gbp,gross_profit_gbp
0,WHSmith,1,6000,61840.50,25511.75
1,WHSmith,2,6244,64620.84,26687.54
2,WHSmith,3,4634,47615.92,19605.50
3,WHSmith,4,4624,46985.09,19233.37
4,WHSmith,5,4598,47475.89,19579.27


In [9]:
yearly_company = (
    sales.groupby("company", as_index=False)
         .agg(units_sold=("units_sold", "sum"),
              revenue_gbp=("revenue_gbp", "sum"),
              gross_profit_gbp=("gross_profit_gbp", "sum"))
)

weekly_avg = (
    weekly_company.groupby("company", as_index=False)
                 .agg(avg_weekly_units=("units_sold", "mean"),
                      avg_weekly_revenue=("revenue_gbp", "mean"),
                      avg_weekly_profit=("gross_profit_gbp", "mean"))
)

yearly_projection = weekly_avg.copy()
yearly_projection["proj_yearly_units"] = yearly_projection["avg_weekly_units"] * 52
yearly_projection["proj_yearly_revenue"] = yearly_projection["avg_weekly_revenue"] * 52
yearly_projection["proj_yearly_profit"] = yearly_projection["avg_weekly_profit"] * 52

yearly_company, yearly_projection


(       company  units_sold  revenue_gbp  gross_profit_gbp
 0      WHSmith      253249   2602809.69        1071875.06
 1  Waterstones      329909   3514844.33        1448091.92,
        company  avg_weekly_units  avg_weekly_revenue  avg_weekly_profit  \
 0      WHSmith       4870.173077        50054.032500       20612.981923   
 1  Waterstones       6344.403846        67593.160192       27847.921538   
 
    proj_yearly_units  proj_yearly_revenue  proj_yearly_profit  
 0           253249.0           2602809.69          1071875.06  
 1           329909.0           3514844.33          1448091.92  )

In [10]:
demo_profit = (
    sales[sales["units_sold"] > 0]
    .groupby(["company", "age_band", "income_band", "customer_type"], as_index=False)
    .agg(units_sold=("units_sold", "sum"),
         revenue_gbp=("revenue_gbp", "sum"),
         gross_profit_gbp=("gross_profit_gbp", "sum"))
    .sort_values(["company", "gross_profit_gbp"], ascending=[True, False])
)

demo_profit.head(15)


,company,age_band,income_band,customer_type,units_sold,revenue_gbp,gross_profit_gbp
44,WHSmith,55-64,Mid,Returning,21589,221984.44,91458.26
35,WHSmith,45-54,Mid,Returning,20128,207038.89,85366.72
43,WHSmith,55-64,Mid,New,16991,174889.21,71939.24
34,WHSmith,45-54,Mid,New,15811,162188.97,66736.59
26,WHSmith,35-44,Mid,Returning,13133,135299.82,55797.04
53,WHSmith,65+,Mid,Returning,12561,128808.75,52954.49
25,WHSmith,35-44,Mid,New,10142,104627.69,43182.78
52,WHSmith,65+,Mid,New,9711,99511.06,41021.52
41,WHSmith,55-64,Low,Returning,9655,99363.56,40993.61
32,WHSmith,45-54,Low,Returning,8898,91440.42,37612.20


In [11]:
sales.to_csv("tom_clancy_england_sales_rowlevel.csv", index=False)
weekly_company.to_csv("tom_clancy_england_weekly_company.csv", index=False)
yearly_company.to_csv("tom_clancy_england_yearly_company.csv", index=False)

print("Saved 3 CSV files in your notebook folder.")


Saved 3 CSV files in your notebook folder.
